In [304]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [228]:
iris = load_iris()
train_dataset = iris.data
new_input=[]

In [286]:
train_dataset = np.genfromtxt('train.csv', dtype=np.float32, delimiter=',')
train_targets = train_dataset[:,-1]
test_dataset = np.genfromtxt('test.csv', dtype=np.float32, delimiter=',')
test_targets = test_dataset[:,-1]
combined_dataset=np.concatenate((train_dataset[:,:-1], test_dataset[:,:-1]))
combined_dataset = MinMaxScaler().fit_transform(combined_dataset)

In [339]:
batch_size = 500
num_feature = 122
num_labels = 5
graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, num_feature))
    tf_test_dataset = tf.constant(combined_dataset)
    
    n_hidden_1 = 2000
#     n_hidden_2 = 100
#     n_hidden_3 = 500
    n_input = num_feature

    wh1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1], dtype=tf.float32))
#     wh2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], dtype=tf.float32))
#     wh3 = tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], dtype=tf.float32))
    wo = tf.Variable(tf.zeros([n_hidden_1, n_input], dtype=tf.float32))
    wo.initialized_value = tf.transpose(wh1)
    b1 = tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32))
#     b2 = tf.Variable(tf.truncated_normal([n_hidden_2], dtype=tf.float32))
#     b3 = tf.Variable(tf.truncated_normal([n_hidden_3], dtype=tf.float32))
    bo = tf.Variable(tf.truncated_normal([n_input], dtype=tf.float32))
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(tf_train_dataset, wh1), b1))
    def compute_logit(x):
#         layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, wh2), b2))
#         layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, wh3), b3))
        return tf.matmul(layer_1, wo) + bo 

    
    logits = compute_logit(tf_train_dataset)
    difference = tf.sub(tf_train_dataset, logits) 
    loss = tf.sqrt(tf.reduce_mean(tf.square(difference)))# +  0.0001 * (tf.nn.l2_loss(wh1) + tf.nn.l2_loss(wo))
#     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_dataset))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    train_prediction = compute_logit(tf_train_dataset)

In [340]:
num_steps = 5000
preds = []
randomized_dataset = combined_dataset.copy()
np.random.shuffle(randomized_dataset)
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)

        batch_data = randomized_dataset[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step", step, ":", l
#             print "SKLEARN loss", np.sqrt(mean_squared_error(predictions, batch_data))
    new_input=layer_1.eval(feed_dict = {tf_train_dataset :combined_dataset})
    new_input1=new_input[:train_dataset.shape[0]]
    new_input2=new_input[train_dataset.shape[0]:]
    print new_input.shape

Initialized
Minibatch loss at step 0 : 0.84512
Minibatch loss at step 500 : 0.0224648
Minibatch loss at step 1000 : 0.0174672
Minibatch loss at step 1500 : 0.0134327
Minibatch loss at step 2000 : 0.0137454
Minibatch loss at step 2500 : 0.0162045
Minibatch loss at step 3000 : 0.0143678
Minibatch loss at step 3500 : 0.0151328
Minibatch loss at step 4000 : 0.0109258
Minibatch loss at step 4500 : 0.0107216
(148517, 2000)


In [9]:
model = LogisticRegression(C=1)

In [256]:
print -cross_val_score(model, train_dataset[:,:-1], train_targets, cv=3, scoring='log_loss').mean()

0.0767538104459


In [260]:
print cross_val_score(model, train_dataset[:,:-1], train_targets, cv=3, scoring='precision_macro')

[ 0.8367046   0.90368039  0.91792525]


In [263]:
print -cross_val_score(model, preds, train_targets, cv=3, scoring='log_loss').mean()

0.0767401043007


In [143]:
print -cross_val_score(model, new_input, iris.target, cv=10, scoring='log_loss').mean()

0.0732442215694


In [299]:
print -cross_val_score(model, new_input1, train_targets, cv=3, scoring='log_loss').mean()

0.0377882059865


In [ ]:
model.fit(new_input1, train_targets)

In [322]:
preds = model.predict(new_input2)

In [323]:
accuracy_score(preds, test_targets)

0.43075762952448543

In [324]:
confusion_matrix(test_targets, preds)

array([[   0, 7458,    0,    0,    0],
       [   0, 9711,    0,    0,    0],
       [   0, 2421,    0,    0,    0],
       [   0, 2554,    0,    0,    0],
       [   0,  400,    0,    0,    0]])